In [41]:
import networkx as nx
import os
from scipy.io import mmread
import numpy as np

import random


In [37]:
def load_graph(filepath: str) -> nx.Graph:
    """
    Carrega grafos de arquivos .txt, .edges, .edge, .csv, .gml ou mtx.
    Detecta automaticamente:
    1. Delimitadores: Espaço, Vírgula (CSV) ou Tabulação.
    2. Comentários: '#' ou '%'.
    3. Cabeçalhos de texto (ex: id_1, id_2) e os remove.
    """
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"Arquivo não encontrado: {filepath}")

    # Se for GML, usa o leitor específico
    if filepath.endswith('.gml'):
        return nx.read_gml(filepath, label='id')
    
    if filepath.endswith(".mtx"):
        sparse_matrix = mmread(filepath)
        G = nx.from_scipy_sparse_array(sparse_matrix)
        g = G.to_undirected()
        G.remove_edges_from(nx.selfloop_edges(G))
        return G


    # Configurações padrão
    delimiter = None 
    comment_char = '#'
    has_header = False # Nova flag para controle
    
    # 1. ESPIAR O ARQUIVO (Detecção de metadados e cabeçalho)
    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            if not line: continue 
            
            # Se for comentário explícito, detectamos o caractere e seguimos
            if line.startswith(('%', '#')):
                comment_char = line[0]
                continue
            
           
            # Detecta separador
            if ',' in line: delimiter = ','
            elif '\t' in line: delimiter = '\t'
            else: delimiter = None # Espaço

            # Teste de conversão: Tenta converter o primeiro elemento para int
            parts = line.split(delimiter) if delimiter else line.split()
            try:
                int(parts[0]) # Se conseguir virar número, é dado!
                has_header = False
            except ValueError:
                has_header = True
            
            break 

    print(f"Lendo {filepath} | Delim: '{'espaço' if delimiter is None else delimiter}' | Header: {has_header}")

    # 2. LER O ARQUIVO (Pulando o cabeçalho se necessário)
    try:
        with open(filepath, 'r') as f:
            if has_header:
                next(f)
            G = nx.read_edgelist(
                f, 
                nodetype=int,          
                comments=comment_char, 
                delimiter=delimiter,   
                create_using=nx.Graph(), 
                data=False             
            )
        
        G.remove_edges_from(nx.selfloop_edges(G))
        return G
        
    except Exception as e:
        raise ValueError(f"Erro crítico ao ler {filepath}: {e}")
 

In [38]:
def calc_req(grafo:nx.Graph):
    for node, grau in grafo.degree():
        grafo.nodes[node]["requisito"] =  (grau+1)//2

       

In [5]:
dir_path = "../data"
arquivos_instancias = os.listdir("../data")
files = []
instancias = []
for file in arquivos_instancias:
    full_path = os.path.join(dir_path, file)
    if os.path.isfile(full_path):
        nome, base = os.path.splitext(file)
        G = load_graph(full_path)
        instancias.append(G)



for Grafo in instancias:
    calc_req(Grafo)


Lendo ../data/email-Enron.txt | Delim: '	' | Header: False
Lendo ../data/ca-GrQc.txt | Delim: '	' | Header: False
Lendo ../data/ca-AstroPh.txt | Delim: '	' | Header: False
Lendo ../data/ca-HepTh.txt | Delim: '	' | Header: False
Lendo ../data/ca-HepPh.txt | Delim: '	' | Header: False
Lendo ../data/ca-CondMat.txt | Delim: '	' | Header: False
Lendo ../data/socfb-nips-ego.edges | Delim: 'espaço' | Header: False
Lendo ../data/soc-gplus.edges | Delim: 'espaço' | Header: False
Lendo ../data/musae_git_edges.csv | Delim: ',' | Header: True
Lendo ../data/loc-gowalla_edges.txt | Delim: '	' | Header: False
Lendo ../data/artist_edges.csv | Delim: ',' | Header: True
Lendo ../data/HR_edges.csv | Delim: ',' | Header: True
Lendo ../data/com-dblp.ungraph.txt | Delim: '	' | Header: False
Lendo ../data/amazon0302.txt | Delim: '	' | Header: False
Lendo ../data/amazon0312.txt | Delim: '	' | Header: False
Lendo ../data/amazon0505.txt | Delim: '	' | Header: False
Lendo ../data/amazon0601.txt | Delim: '	' | He

In [85]:
class GraphInstance:
    def __init__(self, filepath:str):
        print(f"Carregando grafo: {filepath}...")
        self.path = filepath
        print(self.path)
        self.graph = self.load_graph()
        self.reqs = []
        self.calc_req()
        self.sol = np.zeros(shape=(len(self.graph.nodes)), dtype=np.int32)
        self.adj_mat = nx.adjacency_matrix(self.graph)
        self.active= np.zeros(shape=(len(self.graph.nodes)), dtype=np.int32)

    def load_graph(self):
        """
        Carrega grafos de arquivos .txt, .edges, .edge, .csv, .gml ou mtx.
        Detecta automaticamente:
        1. Delimitadores: Espaço, Vírgula (CSV) ou Tabulação.
        2. Comentários: '#' ou '%'.
        3. Cabeçalhos de texto (ex: id_1, id_2) e os remove.
        """
        if not os.path.exists(self.path):
            raise FileNotFoundError(f"Arquivo não encontrado: {self.path}")

        # Se for GML, usa o leitor específico
        if self.path.endswith('.gml'):
            return nx.read_gml(self.path, label='id')
        
        if self.path.endswith(".mtx"):
            sparse_matrix = mmread(self.path)
            G = nx.from_scipy_sparse_array(sparse_matrix)
            g = G.to_undirected()
            G.remove_edges_from(nx.selfloop_edges(G))
            return G


        # Configurações padrão
        delimiter = None 
        comment_char = '#'
        has_header = False # Nova flag para controle
        
        # 1. ESPIAR O ARQUIVO (Detecção de metadados e cabeçalho)
        with open(self.path, 'r') as f:
            for line in f:
                line = line.strip()
                if not line: continue 
                
                # Se for comentário explícito, detectamos o caractere e seguimos
                if line.startswith(('%', '#')):
                    comment_char = line[0]
                    continue
                
            
                # Detecta separador
                if ',' in line: delimiter = ','
                elif '\t' in line: delimiter = '\t'
                else: delimiter = None # Espaço

                # Teste de conversão: Tenta converter o primeiro elemento para int
                parts = line.split(delimiter) if delimiter else line.split()
                try:
                    int(parts[0]) # Se conseguir virar número, é dado!
                    has_header = False
                except ValueError:
                    has_header = True
                
                break 

        print(f"Lendo {self.path} | Delim: '{'espaço' if delimiter is None else delimiter}' | Header: {has_header}")

        # 2. LER O ARQUIVO (Pulando o cabeçalho se necessário)
        try:
            with open(self.path, 'r') as f:
                if has_header:
                    next(f)
                G = nx.read_edgelist(
                    f, 
                    nodetype=int,          
                    comments=comment_char, 
                    delimiter=delimiter,   
                    create_using=nx.Graph(), 
                    data=False             
                )
            
            G.remove_edges_from(nx.selfloop_edges(G))
            return G
            
        except Exception as e:
            raise ValueError(f"Erro crítico ao ler {self.path}: {e}")
        
    def calc_req(self):
        for node, grau in self.graph.degree():
            req = (grau+1)//2
            self.graph.nodes[node]["requisito"] = req
            self.reqs.append(req)

    def step(self, no):
        self.sol[no] = 1
        self.propagate()

    def propagate(self):
        active = self.sol.copy().astype(np.float32) 
        changed = True
        while changed:
            neigh_sum = self.adj_mat @ active
            to_active = (neigh_sum >= self.reqs).astype(np.float32)
            new_active = np.maximum(active, to_active)
            changed = bool(np.any(new_active != active))
            active = new_active
            print(active.sum())
        self.active = active
        


In [27]:
sorted_list = sorted(grafo.nodes, key=lambda x:grafo.nodes[x]['requisito'], reverse=True)

grafo.nodes[sorted_list[2]]

{'requisito': 631}

In [ ]:
from typing import List, Set
def guloso(Instancia:GraphInstance, alpha = 0.8):
    solution: List[int] = []
    max_it = len(Instancia.sol)
    ordered_nodes = sorted(Instancia.graph.nodes, key=lambda x:Instancia.graph.nodes[x]['requisito'], reverse=True)
    while Instancia.active.sum() < max_it:
        cl = [node for node in ordered_nodes if Instancia.sol[node] == 0]
        req_limite = int(Instancia.reqs[cl[-1]]+ alpha * (Instancia.reqs[cl[0]] - Instancia.reqs[cl[-1]]))
        rcl = [no for no in cl if Instancia.reqs[no] >= req_limite]
        pick = random.choice(rcl)
        Instancia.step(pick)
        print(f"No adicionado: {pick} | nos ativos: {Instancia.active.sum()}")







Carregando grafo: ../data/football.gml...
../data/football.gml
1.0


In [87]:
grafo_inst.active

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

[6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 5,
 5,
 5,
 6,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 5,
 6,
 5,
 6,
 5,
 6,
 6,
 6,
 6,
 5,
 6,
 6,
 4,
 6,
 6,
 6,
 6,
 5,
 4,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 5,
 6,
 5,
 6,
 5,
 6,
 5,
 5,
 5,
 4,
 6,
 6,
 6,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 5,
 6,
 6,
 6,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 5,
 6,
 6,
 6,
 6,
 5,
 6,
 6,
 5,
 5,
 5,
 5,
 4,
 6,
 5,
 6,
 5,
 5,
 5,
 6,
 5,
 6,
 5,
 5,
 6,
 6,
 6,
 5,
 5,
 6]

In [ ]:
path = "../data/football.gml"
grafo_inst = GraphInstance(path)
guloso(grafo_inst)